<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/TestApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bert-extractive-summarizer
!pip install -U transformers

     |████████████████████████████████| 2.0MB 6.1MB/s 
     |████████████████████████████████| 890kB 22.9MB/s 
     |████████████████████████████████| 3.2MB 15.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=280162c33712d2c5ca87d16c2d9777aacc6aa351387b16474f12930b9844bb44
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)


In [2]:
import requests
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from tqdm import tqdm
from summarizer import Summarizer

import pickle
import re
import math

In [ ]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
#custom_model.to('cuda')
custom_model.eval()
#torch.cuda.get_device_name(0)

In [5]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [19]:
with open("/content/TestDoc", 'r') as f:
    text = f.read()
    text = "".join([i.replace('\n',' ').replace('\t',' ') for i in text])
text
embeddings = modelSum.run_embeddings(text, num_sentences=10)

'Пермский филиал федерального государственного автономного образовательного учреждения высшего образования  «Национальный исследовательский университет  «Высшая школа экономики»   Факультет экономики, менеджмента и бизнес-информатики   Абросимова Полина Сергеевна   РАЗРАБОТКА СРЕДСТВ АВТОМАТИЗАЦИИ РАСШИРЕНИЯ ОНТОЛОГИИ НА ОСНОВЕ ДАННЫХ ИНТЕРНЕТ-ИСТОЧНИКОВ  Выпускная квалификационная работа  по направлению подготовки 38.03.05\xa0Бизнес-информатика образовательная программа «Бизнес-информатика»      К.ф.-м.н., доцент, ПГНИУ,    Руководитель доцент кафедры       к.ф.-м.н, доцент, доцент информационных технологий    кафедры Л.А. Залогова      информационных         технологий в бизнесе         __________________________         Л.Н. Лядова             Пермь, 2019 год Аннотация В данной работе совершенствуется система мониторинга глобальных процессов сети Интернет. Приводится обоснование необходимости пополнения существующей онтологии, а также ведётся анализ трёх основных методов пополнения 

In [20]:
with open("/content/ЛядоваЛюдмилаНиколаевна.data", "rb") as f:
    arr = pickle.load(f)
arr.shape

(1285, 768)

In [22]:
%time embeddings = modelSum.run_embeddings(text, num_sentences=10)

CPU times: user 1min 9s, sys: 2.54 s, total: 1min 12s
Wall time: 1min 11s


In [24]:
embeddings.shape

(11, 768)

In [25]:
embeddings[0][100]

1.5039678

In [33]:
sum = 0
avg = 0
mae = 0
rmae = 0
mse = 0
rmse = 0
for i in tqdm(range(11)):
    for arrRow in range(len(arr)):
        for j in range(768):
            sum += embeddings[i][j] - arr[arrRow][j]
            avg += 1
            mae += abs(embeddings[i][j] - arr[arrRow][j])
            rmae += abs(embeddings[i][j] - arr[arrRow][j])**0.5
            mse += (embeddings[i][j] - arr[arrRow][j])**2
            rmse += ((embeddings[i][j] - arr[arrRow][j])**2)**0.5
print(f"Count = {avg}")
print(f"Sum = {sum}")
print(f"Avg = {sum/avg}")
print(f"MAE = {mae/avg}")
print(f"RMAE = {rmae/avg}")
print(f"MSE = {mse/avg}")
print(f"RMSE = {rmse/avg}")

100%|██████████| 11/11 [02:53<00:00, 15.74s/it]

Count = 10855680
Sum = 861.9069203189574
Avg = 7.939686139596574e-05
MAE = 0.42284764881514914
RMAE = 0.595303441929663
MSE = 0.30424415837365826
RMSE = 0.42284764881514914


In [31]:
len(arr)

1285